In [1]:
import json
import pandas as pd
import ast
import datetime
from misc import display, convert_unixtime, convert_country_code, cut, copy_rename, get_slice, calculate_percentages

In [2]:
#infile = "collections/immigrants_global/dataframe_collected_finished_1537157689.csv.gz"
#infile = "collections/immigrants_global/dataframe_collected_finished_1537285977.csv.gz"
# ... missed some...
#infile = "collections/immigrants_global/dataframe_collected_finished_1537764161.csv.gz"

#infile = "collections/immigrants/dataframe_collected_finished_1537725217.csv.gz"
# ... missed some...
#infile = "collections/immigrants/dataframe_collected_finished_1538749138.csv.gz"
# ... missed some...
#infile = "collections/immigrants/dataframe_collected_finished_1539857988.csv.gz"
# ... missed some...
#infile = "collections/immigrants/dataframe_collected_finished_1539857988.csv.gz"
#infile = "collections/immigrants/dataframe_collected_finished_1541491288.csv.gz"
#infile = "collections/immigrants/dataframe_collected_finished_1541590184.csv.gz"
infile = "collections/immigrants/dataframe_collected_finished_1542039176.csv.gz"

df = pd.read_csv(infile)

#syrians = "collections/syrians/dataframe_collected_finished_1537176338.csv.gz"
#syrians = "collections/syrians/dataframe_collected_finished_1537343532.csv.gz"
#syrians = "collections/syrians/dataframe_collected_finished_1537345011.csv.gz"
# ... missied some...
#syrians = "collections/syrians/dataframe_collected_finished_1537783123.csv.gz"
#sdf = pd.read_csv(syrians)
#df = pd.concat((sdf,df))

day, month = df["timestamp"].apply(lambda x: convert_unixtime(x)).head(1).values[0].split("-")
print("Collection made on %s-%s" % (day, month))



Collection made on 12-11


In [3]:
def extract_relationship(d):
    if 1 in d:
        return "single"
    elif 2 in d:
        return "dating"
    elif 3 in d:
        return "married"
    else:
        return None

def extract_education(d):
    if d == [3, 7, 8, 9, 11]:
        return "graduated"
    elif d == [1, 12, 13]:
        return "no_degree"
    elif d == [2, 4, 5, 6, 10]:
        return "high_school"
    elif d == [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
        return "all"
    else:
        return None

def extract_group(d):
    id = d[0]["id"]
    if id == 6026404871583:
        return "Expats (Venezuela)"
    elif id == 6015559470583:
        return "Ex-pats (All)"
    else:
        return "All"

def agebuckets(minage, maxage):
    if minage == 13 and (maxage is None or np.isnan(maxage)):
        return "all"
    elif minage == 13 and maxage == 18:
        return "adolecent"
    elif minage == 19 and maxage == 25:
        return "young_adult"
    elif minage == 26 and maxage == 40:
        return "adult"
    elif minage == 41 and maxage == 65:
        return "middle_age"
    elif minage == 65 and (maxage is None or np.isnan(maxage)):
        return "elder"
    return "undefined"
    
def expand(row):
    place = None
    loc_dimension = None
    if "regions" in row["geo_locations"]:
        place = "%s, %s" % (row["geo_locations"]["regions"][0]["name"], convert_country_code(row["geo_locations"]["regions"][0]["country_code"]))
        loc_dimension = "State"
    elif "countries" in row["geo_locations"]:
        place = convert_country_code(row["geo_locations"]["countries"][0])
        loc_dimension = "Country"
    elif "cities" in row["geo_locations"]:
        place = "%s, %s, %s" % (row["geo_locations"]["cities"][0]["name"], row["geo_locations"]["cities"][0]["region"], convert_country_code(row["geo_locations"]["cities"][0]["country"]))
        loc_dimension = "City"
    
    loctype = "_".join(row["geo_locations"]["location_types"])

    relationship, education, group = None, None, None
    for dimension in row["flexible_spec"]:
        if "relationship_statuses" in dimension:
            relationship = extract_relationship(dimension["relationship_statuses"])
        elif "education_statuses" in dimension:
            education = extract_education(dimension["education_statuses"])
        elif "behaviors" in dimension:
            group = extract_group(dimension["behaviors"])
    
    gender = row["genders"][0]
    gender = "both" if gender == 0 else "man" if gender == 1 else "woman"
    
    return row["age_min"], row["age_max"], place, loc_dimension, loctype, gender, relationship, education, group

def get_item(x):
    if not x:
        return None
    return x["name"]

df[["MinAge","MaxAge","Location","LocationHierarchy","LocationType","Gender","Relationship","Education","Group"]] = df["targeting"].apply(lambda x : expand(ast.literal_eval(x))).apply(pd.Series)

if "citizenship" in df:
    df["Group"] = df["citizenship"].fillna("[]").apply(lambda x : get_item(ast.literal_eval(x)))
    print("Updating Group information...")
    
if "access_device" in df:
    df["Device"] = df["access_device"].fillna("[]").apply(lambda x : get_item(ast.literal_eval(x)))
    print("Adding information regarding devices...")
else:
    df["Device"] = None
    
df["AgeBucket"] = df[["MinAge","MaxAge"]].apply(lambda x: agebuckets(x["MinAge"], x["MaxAge"]), axis=1)

print("Removing redundant cols")
for col in ["Unnamed: 0", "all_fields", "targeting",'behavior', 'citizenship', "mock_response", "access_device", 
            "ages_ranges", "household_composition", 'interests', 'family_statuses', 'genders', 'geo_locations', 'languages',
            'name', 'relationship_statuses', 'response', 'scholarities', 'timestamp', 'publisher_platforms',]:
    if col in df.keys():
        del df[col]

# Gave up using travel_in   
df = df[df["LocationType"] != "travel_in"]

print("All Done!")
# Brief description: 
# -----------------
# Gender: 0 Both, 1 Man, 2 Woman
# Age: 13-Null, 13-18, 19-25, 26-40, 41-65, 65+
# LocationType: home_recent, home, recent


Updating Group information...
Removing redundant cols
All Done!


In [4]:
dfgender = {}
dfcut = df[(df["Gender"].apply(lambda x : x in ["man","woman"])) & (df["Education"].isnull()) & (df["Device"].isnull()) & (df["AgeBucket"] == "all") & (df["Group"] == "Expats (Venezuela)") & (df["LocationType"] == "home_recent")].copy()
dfgender["mau"] = get_slice(dfcut, "Gender", ["man","woman"], frequency="mau")
dfgender["dau"] = get_slice(dfcut, "Gender", ["man","woman"], frequency="dau")
calculate_percentages(dfgender["mau"], ["man","woman"], "%")
calculate_percentages(dfgender["dau"], ["man","woman"], "%")

In [5]:
df_gender_locals = {}
dfcut = df[(df["Gender"].apply(lambda x : x in ["man","woman"])) & (df["Education"].isnull()) & (df["Device"].isnull()) 
           & (df["AgeBucket"] == "all") & (df["Group"].isnull()) & (df["LocationType"] == "home_recent") ].copy()

df_gender_locals["mau"] = get_slice(dfcut, "Gender", ["man","woman"], frequency="mau") 
df_gender_locals["dau"] = get_slice(dfcut, "Gender", ["man","woman"], frequency="dau")

calculate_percentages(df_gender_locals["mau"], ["man","woman"], "%locals_")
calculate_percentages(df_gender_locals["dau"], ["man","woman"], "%locals_")

for col in ["man","woman"]:
    del df_gender_locals["mau"]["audience_" + col]
    del df_gender_locals["dau"]["audience_" + col]


In [6]:
def get_top_going_to_aux(n, df, group):
    
    colname = 'audience_Expats (' + group + ')'
    
    if colname in df:
        idx = df[colname].nlargest(n).index
        locations = df.iloc[idx][["Location"]].T
        values = df.iloc[idx][[colname]].T
    else:
        locations = pd.DataFrame(['-'] * nlargest, index=range(1, nlargest+1)).T
        values = pd.DataFrame(['-'] * nlargest, index=range(1, nlargest+1)).T

    locations.index = [group]
    values.index = [group]
    return locations, values

def get_top_going_to(n, df):
    acc_loc, acc_val = [], []
    for loc in  df_immi_place["mau"]["Location"]:
        toAppend_loc, toAppend_val = get_top_going_to_aux(nlargest, df, loc)
        toAppend_loc.columns = ['goingTo{}'.format(i) for i in range(1, nlargest+1)]
        toAppend_val.columns = ['goingTo{}_value'.format(i) for i in range(1, nlargest+1)]
        
        
        acc_loc.append(toAppend_loc)
        acc_val.append(toAppend_val)

    
    toAppend_val = pd.concat(acc_val)
    toAppend_loc = pd.concat(acc_loc)
    return toAppend_val, toAppend_loc

In [7]:

immigrant_groups = ["Expats (Philippines)", "Expats (Cuba)", "Expats (Ethiopia)", "Expats (Haiti)", "Expats (Spain)", "Expats (France)", "Expats (Germany)", "Expats (Switzerland)", "Expats (Hungary)", 
                    "Expats (United States)", "Expats (Ireland)", "Expats (Italy)", "Expats (Poland)", "Expats (Canada)", "Expats (China)", "Expats (Puerto Rico)", "Expats (Brazil)", "Expats (Indonesia)",
                    "Expats (South Africa)", "Expats (Zimbabwe)", "Expats (Ghana)", "Expats (Uganda)", "Expats (Colombia)", "Expats (Dominican Republic)", "Expats (El Salvador)", "Expats (Guatemala)", 
                    "Expats (United Kingdom)", "Expats (Australia)", "Expats (Portugal)", "Expats (Estonia)", "Expats (Netherlands)", "Expats (Sweden)", "Expats (Czech Republic)",  "Expats (Denmark)", 
                    "Expats (Norway)", "Expats (Bangladesh)", "Expats (Tanzania)", "Expats (Nepal)", "Expats (Jamaica)", "Expats (Thailand)", "Expats (Sierra Leone)", "Expats (Senegal)", "Expats (Cote d'Ivore)", 
                    "Expats (Sri Lanka)", "Expats (Morocco)", "Expats (New Zealand)", "Expats (Congo Dem. Rep.)",  "Expats (Singapore)", "Expats (United Arab Emirates)", "Expats (Austria)", "Expats (Cyprus)", 
                    "Expats (Greece)", "Expats (Hong Kong)", "Expats (Japan)", "Expats (Lithuania)", "Expats (Luxembourg)", "Expats (Malta)", "Expats (Monaco)", "Expats (Slovakia)", "Expats (Slovenia)", "Expats (Mexico)",
                    "Expats (Saudi Arabia)", "Expats (Russia)", "Expats (Israel)", "Expats (Argentina)", "Expats (Chile)", "Expats (Rwanda)", "Expats (Venezuela)", "Expats (Malaysia)", "Expats (Romania)", "Expats (South Korea)",
                    "Expats (Serbia)", "Expats (Vietnam)", "Expats (Peru)", "Expats (Belgium)", "Expats (Zambia)", "Expats (Honduras)", "Expats (Finland)", "Expats (Latvia)", "Expats (Jordan)", "Expats (Lebanon)",
                    "Expats (Nicaragua)", "Expats (Algeria)", "Expats (Kuwait)", "Expats (Qatar)", "Expats (Syria)"
                   ]

df_immi_place = {}
dfcut = df[(df["Group"].apply(lambda x: x in immigrant_groups)) & (df["Gender"] == "both") & (df["Education"].isnull()) & 
           (df["Device"].isnull()) & (df["AgeBucket"] == "all") & (df["LocationType"] == "home_recent")].copy()
df_immi_place["mau"] = get_slice(dfcut, "Group", immigrant_groups, frequency="mau")
df_immi_place["dau"] = get_slice(dfcut, "Group", immigrant_groups, frequency="dau")

calculate_percentages(df_immi_place["mau"], immigrant_groups, "%")
calculate_percentages(df_immi_place["dau"], immigrant_groups, "%")

"""
nlargest = 5

for period in ["dau", "mau"]:
    values, locations = get_top_going_to(nlargest, df_immi_place[period])

    df_immi_place[period] = pd.merge(df_immi_place[period], values, left_on=["Location"], right_index=True)
    df_immi_place[period] = pd.merge(df_immi_place[period], locations, left_on=["Location"], right_index=True)
"""


'\nnlargest = 5\n\nfor period in ["dau", "mau"]:\n    values, locations = get_top_going_to(nlargest, df_immi_place[period])\n\n    df_immi_place[period] = pd.merge(df_immi_place[period], values, left_on=["Location"], right_index=True)\n    df_immi_place[period] = pd.merge(df_immi_place[period], locations, left_on=["Location"], right_index=True)\n'

In [8]:
# For the case of venezuela

immigrant_groups = ['Expats (Venezuela)', u'Expats (United States)', u'Expats (Mexico)', u'Expats (Brazil)', u'Expats (Argentina)', 
                    u'Expats (Peru)', u'Expats (Chile)', u'Expats (Spain)', u'Expats (France)', u'Expats (United Kingdom)', u'Expats (Colombia)', 
                    u'Expats (Cuba)', u'Expats (Germany)', u'Expats (Italy)', u'Expats (Portugal)', u'Expats (Jamaica)', 'Expats (China)']
df_immi_place = {}
dfcut = df[(df["Group"].apply(lambda x: x in immigrant_groups)) & (df["Gender"] == "both") & (df["Education"].isnull()) & 
           (df["Device"].isnull()) & (df["AgeBucket"] == "all") & (df["LocationType"] == "home_recent")].copy()
df_immi_place["mau"] = get_slice(dfcut, "Group", immigrant_groups, frequency="mau")
df_immi_place["dau"] = get_slice(dfcut, "Group", immigrant_groups, frequency="dau")

calculate_percentages(df_immi_place["mau"], immigrant_groups, "%")
calculate_percentages(df_immi_place["dau"], immigrant_groups, "%")



In [9]:
dfcut = df[df["Group"].isnull()]

loc_mau = dfcut[["Location", "mau_audience"]].rename(columns={"mau_audience": "audience"})
loc_dau = dfcut[["Location", "dau_audience"]].rename(columns={"dau_audience": "audience"})

loc_mau["Frequency"] = "Monthly"
loc_dau["Frequency"] = "Daily"

df_countrypop = pd.concat((loc_mau,loc_dau))


In [10]:
# Generated dffinal
dffinal = pd.concat([df_immi_place["mau"], df_immi_place["dau"]])
dffinal = pd.merge(dffinal, df_countrypop, on=["Location","Frequency"])


In [11]:
# Add aggregated stats for total number of immigrants
audience_group = []
for group in immigrant_groups:
    audience_group += ["audience_" + group]
    
dffinal["summary_total_immigrants"] = dffinal[audience_group].sum(axis=1)        
dffinal["immigrants_in_population"] = (100. * dffinal["summary_total_immigrants"] / dffinal["audience"])
dffinal["immigrants_in_population"] = dffinal["immigrants_in_population"].apply(lambda x: "%.1f%%" % x)

dffinal["display_total_immigrants"] = dffinal["summary_total_immigrants"].fillna(0).apply(lambda x: display(x))


In [12]:
nlargest = 5

for col in dffinal.keys():
    if col.startswith("%"):
        #print(col)
        dffinal[col] = dffinal[col].round(2)

# Add TOP 'nlargest' population counts:
audiences = [k for k in dffinal.keys() if k.startswith("audience_")]
dfaud = dffinal[audiences]

def cleanCoutryName(name):
    return " ".join(name.rsplit()[1:]).strip("()")

order = np.argsort(-dfaud.values, axis=1)[:, :nlargest]
names = pd.DataFrame(dfaud.columns[order], columns=['comeFrom{}'.format(i) for i in range(1, nlargest+1)],
                      index=dfaud.index)

for k in names.keys():
    names[k] = names[k].apply(lambda x: cleanCoutryName(x))

values = np.abs(np.sort(-dfaud.values, axis=1)[:,:nlargest])
results = pd.DataFrame(values, columns=['comeFrom{}_value'.format(i) for i in range(1, nlargest+1)],
                      index=dfaud.index)

dffinal = pd.concat((dffinal, names, results), axis=1)


In [13]:
#dffinal.to_csv("global_%s_%s.csv" % (month, day), index=False)
dffinal.to_csv("immigrants_%s_%s.csv" % (month, day), index=False)
print("Saved immigrants_%s_%s.csv" % (month, day))


Saved immigrants_11_12.csv
